In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
from dask.distributed import Client

np.set_printoptions(precision=4, suppress=True)

In [ ]:
# ds = xr.load_dataset("../data/outputs/plots/metrics/")